<a href="https://colab.research.google.com/github/tnewtont/ModCloth_Recommendation_System/blob/main/rsp_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [3]:
def read_and_filter_data(filepath):
    # Obain the dataframe
    df = pd.read_csv(filepath)

    # We have a null in user_id, so impute it
    df['user_id'] = df['user_id'].fillna('Unknown')

    # Get the necessary columns
    ratings = df[['item_id', 'user_id', 'rating', 'category']]

    # Get the value counts of the items
    vc_items = ratings['item_id'].value_counts()

    # Filter out items that don't have 24 or more instances
    vc_items_24 = vc_items[vc_items >= 24].index

    # Apply this filter to our data
    ratings_filtered = ratings.loc[ratings['item_id'].isin(vc_items_24)]

    return ratings_filtered

First, we are only going to consider four features: <br>
(1) item_id<br>
(2) user_id<br>
(3) rating<br>
(4) category<br>

In [5]:
# Loading the data
df = pd.read_csv('/content/df_modcloth.csv')
df

,item_id,user_id,rating,timestamp,size,fit,user_attr,model_attr,category,brand,year,split
0,7443,Alex,4,2010-01-21 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,NaN,2012,0
1,7443,carolyn.agan,3,2010-01-27 08:00:00+00:00,NaN,NaN,NaN,Small,Dresses,NaN,2012,0
2,7443,Robyn,4,2010-01-29 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,NaN,2012,0
3,7443,De,4,2010-02-13 08:00:00+00:00,NaN,NaN,NaN,Small,Dresses,NaN,2012,0
4,7443,tasha,4,2010-02-18 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,NaN,2012,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99888,154797,BernMarie,5,2019-06-26 21:15:13.165000+00:00,6.0,Just right,Large,Small&Large,Dresses,NaN,2017,0
99889,77949,Sam,4,2019-06-26 23:22:29.633000+00:00,4.0,Slightly small,Small,Small&Large,Bottoms,NaN,2014,2
99890,67194,Janice,5,2019-06-27 00:20:52.125000+00:00,NaN,Just right,Small,Small&Large,Dresses,NaN,2013,2
99891,71607,amy,3,2019-06-27 15:45:06.250000+00:00,NaN,Slightly small,Small,Small&Large,Outerwear,Jack by BB Dakota,2016,2


In [7]:
# Filtering the data to include the 4 features
df2 = df[['item_id', 'user_id', 'rating', 'category']]
df2

,item_id,user_id,rating,category
0,7443,Alex,4,Dresses
1,7443,carolyn.agan,3,Dresses
2,7443,Robyn,4,Dresses
3,7443,De,4,Dresses
4,7443,tasha,4,Dresses
...,...,...,...,...
99888,154797,BernMarie,5,Dresses
99889,77949,Sam,4,Bottoms
99890,67194,Janice,5,Dresses
99891,71607,amy,3,Outerwear


From our earlier EDA, in this particular case, we need to impute 'user_id' since it has 1 missing value. We will simply fill it with 'Unknown'

In [8]:
df2['user_id'] = df2['user_id'].fillna('Unknown')

<ipython-input-8-9a32d8d3fe57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['user_id'] = df2['user_id'].fillna('Unknown')


In [9]:
# Checking that the entire dataframe has no nulls
df2.isna().sum().sum()

0

Now, we need to take into account of the following:<br>
(1) There are much more items rated 4 or above.<br>
(2) The number of reviews a user has given varies.<br>
(3) The number of reviews that a product has varies.<br>
 - This means that we need to define what is considered a "good" item.<br>
 - We found out from our EDA that the median number of reviews that an item has is 17. Specifically, for items that are rated 4 or above, the median is 22. Therefore, we define a "good" item to be one that has at least 24 reviews (this number is chosen a bit arbitrarily, but is sufficient for our implementation) and is rated 4 or above.<br>



In [14]:
# Filter our data to include items that have 24 or more reviews

# Obtain the value counts for each item
vc_items = df2['item_id'].value_counts()

# Include only 24 or more reviews
vc_items_24 = vc_items[vc_items >= 24].index

# Apply this filter to our original data
df2_filtered = df2.loc[df2['item_id'].isin(vc_items_24)]
df2_filtered

,item_id,user_id,rating,category
0,7443,Alex,4,Dresses
1,7443,carolyn.agan,3,Dresses
2,7443,Robyn,4,Dresses
3,7443,De,4,Dresses
4,7443,tasha,4,Dresses
...,...,...,...,...
99888,154797,BernMarie,5,Dresses
99889,77949,Sam,4,Bottoms
99890,67194,Janice,5,Dresses
99891,71607,amy,3,Outerwear


In [41]:
# Saving the filtered dataframe
df2_filtered.to_csv('df_modcloth_filtered.csv', index = False)

In [42]:
# Reading the filtered dataframe
pd.read_csv('/content/df_modcloth_filtered.csv')

,item_id,user_id,rating,category
0,7443,Alex,4,Dresses
1,7443,carolyn.agan,3,Dresses
2,7443,Robyn,4,Dresses
3,7443,De,4,Dresses
4,7443,tasha,4,Dresses
...,...,...,...,...
93910,154797,BernMarie,5,Dresses
93911,77949,Sam,4,Bottoms
93912,67194,Janice,5,Dresses
93913,71607,amy,3,Outerwear


Another thing to consider is that since we will be using collaborative filtering (i.e. user-to-user) for our recommendation system, we need to consider two cases:<br>
(1) The user's average rating is 4 or above. <br>
(2) Otherwise, the user's average rating is below 4. <br>
This is important because we only want our recommendation system to recommend good products. That is, when we apply collaborative filtering, users will get matched with other users with similar product ratings. We do not want this to happen for users whose average product rating is below 4.<br>
Therefore, for users whose average rating is below 4, we will give them generic recommendations (i.e. the top 3 items with the most ratings and are rated 4 or above).

In [16]:
# Obtaining the dataframe that contains generic recommendations

mean_item_ratings = df2.groupby('item_id')['rating'].mean()
mean_item_ratings2 = df2_filtered.groupby('item_id').agg(
        mean = ('rating', 'mean'),
        num_reviews = ('item_id', 'count'),
        cat = ('category', 'min')
    )

In [17]:
pop_items = mean_item_ratings2.sort_values('mean', ascending = False).sort_values('num_reviews', ascending = False)
pop_items

,mean,num_reviews,cat
item_id,,,
34935,4.482247,1887,Tops
21296,4.171760,1636,Bottoms
32405,4.325829,1599,Dresses
32406,4.328648,1494,Dresses
32403,4.367199,1378,Dresses
...,...,...,...
153801,4.083333,24,Bottoms
138414,4.000000,24,Tops
153397,3.791667,24,Bottoms


We want to now give more importance for items that are rated 4 or above than those rated below 4. We can create a feature that multiplies mean and num_reviews together and call it weighted_vals. Moreover,<br>
(1) If an item is rated 4 or above, multiply the corresponding weighted_vals by 500<br>
(2) Otherwise, if an item is rated below 4, divide the corresponding weighted_vals by 500

In [18]:
pop_items['weighted_vals'] = (pop_items['mean'].apply(lambda x: np.round(x/500) if x < 4 else np.round(x *500))) * pop_items['num_reviews']
pop_items

,mean,num_reviews,cat,weighted_vals
item_id,,,,
34935,4.482247,1887,Tops,4228767.0
21296,4.171760,1636,Bottoms,3412696.0
32405,4.325829,1599,Dresses,3458637.0
32406,4.328648,1494,Dresses,3233016.0
32403,4.367199,1378,Dresses,3009552.0
...,...,...,...,...
153801,4.083333,24,Bottoms,49008.0
138414,4.000000,24,Tops,48000.0
153397,3.791667,24,Bottoms,0.0


By dividing the weighted_vals by 500 for items rated below 4, we collapse the value to 0.

In [24]:
# Save this dataframe of generic recommendations
pop_items.to_csv('pop_items.csv')

In [38]:
# Double checking this dataframe was saved properly
pop_items_read = pd.read_csv('/content/pop_items.csv')
pop_items_read.set_index('item_id', inplace = True)
pop_items_read

,mean,num_reviews,cat,weighted_vals
item_id,,,,
34935,4.482247,1887,Tops,4228767.0
21296,4.171760,1636,Bottoms,3412696.0
32405,4.325829,1599,Dresses,3458637.0
32406,4.328648,1494,Dresses,3233016.0
32403,4.367199,1378,Dresses,3009552.0
...,...,...,...,...
153801,4.083333,24,Bottoms,49008.0
138414,4.000000,24,Tops,48000.0
153397,3.791667,24,Bottoms,0.0
